In [1]:
import pandas as pd
# from sqlalchemy import create_engine
import tqdm
# from config import user, password

In [42]:
import sqlite3
sqlite3.connect('../data/olympics_raw.sqlite')
df = pd.read_csv("../data/Summer Olympic medallists 1896 to 2008 - ALL MEDALISTS.csv")

In [3]:
conn = sqlite3.connect('../data/olympics.db')

In [44]:
#transform the olympic csv data
def transform_olympics(df):
    df = df.loc[3:]
    col_names = df.iloc[0].values
    df.columns = col_names
    df = df.iloc[1:]
    df = df.reset_index().drop('index', axis=1)
    return df

df = transform_olympics(df)

In [ ]:
# send cleaned olympics data to database
df.to_sql(name = 'olympics_raw', con = conn, if_exists='replace', index=False)

In [27]:
# read the data from database to check
pd.read_sql('SELECT * FROM olympics_raw LIMIT 5', conn)

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [ ]:
# !pip install geopy

In [ ]:
# !pip install pycountry-convert

In [ ]:
import pycountry
import geopy

### Creating a df with total medals in each country

In [31]:
def total_medals(df):
    df = pd.get_dummies(df, columns = ['Medal'])
    df_agg = df.groupby('NOC').sum()
    df_agg['total'] = df_agg['Medal_Bronze'] + df_agg['Medal_Silver'] + df_agg['Medal_Gold']
    df_agg.columns = ['Bronze', 'Gold', 'Silver', 'Total']
    return df_agg

total_medals(df).head()

,Bronze,Gold,Silver,Total
NOC,,,,
AFG,1.0,0.0,0.0,1.0
AHO,0.0,0.0,1.0,1.0
ALG,8.0,4.0,2.0,14.0
ANZ,5.0,20.0,4.0,29.0
ARG,88.0,68.0,83.0,239.0


In [ ]:
df_agg.to_sql(name = 'country_agg', con = conn, if_exists='replace', index=True)

In [ ]:
temp = pd.read_sql("SELECT * FROM country_agg", con=conn)

In [ ]:
temp.columns = ['ISO_A3', 'Bronze', 'Gold', 'Silver', 'Total']

In [ ]:
df_agg.to_sql(name = 'country_agg', con = conn, if_exists='replace', index=False)

### Helping becky with local total json

In [ ]:
df_agg.to_json("country_agg.json")

In [ ]:
import json

In [ ]:
address = "../Map view/raw/countries copy.json"
with open(address) as json_file:  
    data2 = json.load(json_file)

In [ ]:
data2['features'][0]

In [ ]:
len(data2['features'])

In [ ]:
for x in range(len(data2['features'])):
    temp = data2['features'][x]
    prop = temp['properties']['ISO_A3']
    try:
        print(prop)
        print(df_agg.loc[prop]['Total'])
        temp['total'] = df_agg.loc[prop]['Total']
    except:
        print(f"---------------{prop}-------------")
        print(0)
        temp['total'] = 0

In [ ]:
data2['features'][0]

In [ ]:
data2.to_json('geo_json_tot.json')

### Creating flags_ref file

In [ ]:
with open('geo_json_total.json', 'w') as outfile:
    json.dump(data2, outfile)

In [ ]:
flags = pd.read_csv("../data/flag_images.csv", index_col=0)

In [ ]:
flags.head()

In [ ]:
country_ref = pd.DataFrame(df['NOC'].unique())

In [ ]:
# country_ref[0].apply(lambda x:)
# country_ref['country_name'] = ''
for i in range(len(country_ref[0])):
    try:
#         print(pycountry.countries.get(alpha_3=i).name)
        country_ref.at[i, 'country_name']= pycountry.countries.get(alpha_3=country_ref[0][i]).name
    except:
        print(country_ref[0][i])
    

In [ ]:
for i in range(country_ref.shape[0]):
    if country_ref['country_name'][i] == '':
        print(country_ref[0][i])
        c = input("Enter country name: ")
        country_ref.at[i, 'country_name']= c
        print(f"You have successfully inputted {country_ref[0][i]} as {country_ref['country_name'][i]}")
    

In [ ]:
country_ref.columns = ['code', 'country_name']

In [ ]:
country_ref.to_sql(name = 'country_ref', con = conn, if_exists='replace', index=False)

In [ ]:
pd.read_sql("SELECT * FROM country_ref", con = conn)

In [ ]:
flags.to_sql(name = 'flags_ref', con = conn, if_exists='replace', index=False)

### Haneen data asks

Is it possible to get the number of
1- male vs female for different sports for each country.  eg: in US in 2000 total gold medals for  66 males , 30 silver, female 40 gold, in 1999...etc  

Also the same as above  (per country) but with different medal types per sport, eg: in US in year 2000 for swimming women won 6 gold, 3 silver and men 4 gold and 3 silver, in year 1999....etc  

3- I was also thinking of adding a slider visualization for the  main page ( because i would only sub variables) so a filter for each sport separately through the years  grouping all medal types and genders per country per sport. Example: In 2000 for swimming: US got 33 gold (15 women, 18 men), 30 silver (20 women, 10 men), Canada got 20 (20 women) gold and 40 bronze (40 men), in year 1999....etc

In [45]:
df.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [56]:
han1 = df[['Edition', 'Gender', 'Discipline', 'Medal']]
pd.get_dummies(han1, columns=['Medal']).groupby(['Edition', 'Gender', 'Discipline']).sum().to_csv("haneen.csv")